In [1]:
# --- AoC 2025 - Day 10: Factory ---

# 0. Configuration and Imports
# -----------------------------------------------------------------------------
import os
import sys
import re
from collections import defaultdict, Counter, deque
import math
import itertools
import heapq
from fractions import Fraction

# Define input filename - typically both parts use the same 'input.txt'
INPUT_FILENAME = "input.txt"

# Set up paths for convenience
NOTEBOOK_DIR = os.getcwd() 


# 1. Load Input Data
# -----------------------------------------------------------------------------
def load_input_data(filename):
    """
    Loads input data from a specified file.
    Assumes the input file is in the same directory as the notebook.
    """
    filepath = os.path.join(NOTEBOOK_DIR, filename)
    try:
        with open(filepath, 'r') as f:
            # Read all lines and strip whitespace from each
            return [line.strip() for line in f.readlines()]
    except FileNotFoundError:
        print(f"Error: Input file '{filename}' not found at '{filepath}'")
        return [] # Return an empty list to prevent further errors

# Load the raw data once for both parts
raw_data = load_input_data(INPUT_FILENAME)

if raw_data:
    print(f"Loaded {len(raw_data)} lines from '{INPUT_FILENAME}'.")
    print(f"First 5 lines: {raw_data[:5]}\n")
else:
    print("No data loaded (or file not found). Operations will rely on test data.\n")


# =============================================================================
# >>> START PART 1 (Solve this first!) <<<
# =============================================================================

# 2. Part 1 Data Preprocessing / Parsing
# -----------------------------------------------------------------------------
def parse_data_part1(data_lines):
    """
    Parses the raw input data for Part 1 and Part 2.
    Each line represents a machine.
    Returns a list of machines. Each machine is a dict:
    {
        'target': [0, 1, 1, 0],  # Binary vector for lights (Part 1)
        'buttons': [[3], [1, 3], ...], # List of lists of affected indices
        'joltage': [3, 5, 4, 7]  # Integer vector for joltage (Part 2)
    }
    """
    print("Parsing data...")
    if not data_lines: 
        return []

    machines = []
    
    # Regex to capture the lights part [ ... ] and the buttons part ( ... )
    
    for line in data_lines:
        if not line: continue
        
        # Split lights section from the rest
        try:
            lights_match = re.search(r'\[([.#]+)\]', line)
            if not lights_match:
                print(f"Warning: No lights found in line: {line}")
                continue
            
            lights_str = lights_match.group(1)
            target_vector = [1 if c == '#' else 0 for c in lights_str]
            
            # Find joltage part in curly braces
            joltage_match = re.search(r'\{([\d,]+)\}', line)
            joltage_vector = []
            if joltage_match:
                joltage_vector = [int(x) for x in joltage_match.group(1).split(',')]
            
            # Find all button definitions (numbers in parens)
            # We look in the substring between lights and joltage (or end)
            start_idx = lights_match.end()
            end_idx = joltage_match.start() if joltage_match else len(line)
            buttons_section = line[start_idx:end_idx]
                
            button_matches = re.findall(r'\(([\d,]+)\)', buttons_section)
            buttons = []
            for btn_str in button_matches:
                indices = [int(x) for x in btn_str.split(',')]
                buttons.append(indices)
                
            machines.append({
                'target': target_vector,
                'buttons': buttons,
                'joltage': joltage_vector
            })
            
        except Exception as e:
            print(f"Error parsing line '{line}': {e}")
            continue

    return machines

parsed_input_part1 = parse_data_part1(raw_data)


# 3. Part 1 Solution Algorithm
# -----------------------------------------------------------------------------
def rref_binary(matrix, vector_b):
    """
    Computes the Reduced Row Echelon Form (RREF) of the augmented matrix [A|b] over GF(2).
    matrix: list of lists (rows), represents A
    vector_b: list (column vector), represents b
    
    Returns: 
    - rref_matrix: The transformed matrix A (list of lists)
    - transformed_b: The transformed vector b
    - pivot_cols: List of column indices that contain a pivot
    """
    rows = len(matrix)
    if rows == 0:
        return [], [], []
    cols = len(matrix[0])
    
    # Create augmented matrix as a list of lists (mutable)
    aug = [row[:] + [b_val] for row, b_val in zip(matrix, vector_b)]
    
    pivot_row = 0
    pivot_cols = []
    
    for c in range(cols):
        if pivot_row >= rows:
            break
            
        # Find a row with a 1 in this column, starting from pivot_row
        row_with_one = -1
        for r in range(pivot_row, rows):
            if aug[r][c] == 1:
                row_with_one = r
                break
        
        if row_with_one != -1:
            # Swap rows
            aug[pivot_row], aug[row_with_one] = aug[row_with_one], aug[pivot_row]
            
            # Eliminate 1s in other rows for this column
            for r in range(rows):
                if r != pivot_row and aug[r][c] == 1:
                    # Row XOR
                    for k in range(c, cols + 1):
                        aug[r][k] ^= aug[pivot_row][k]
            
            pivot_cols.append(c)
            pivot_row += 1
            
    # Separate A and b back
    final_matrix = [row[:cols] for row in aug]
    final_b = [row[cols] for row in aug]
    
    return final_matrix, final_b, pivot_cols

def solve_machine(machine):
    """
    Solves for the minimum button presses for a single machine.
    Ax = b over GF(2).
    A: columns are buttons, rows are lights.
    b: target lights.
    """
    target = machine['target']
    buttons = machine['buttons']
    
    num_lights = len(target)
    num_buttons = len(buttons)
    
    # Construct Matrix A
    # A[i][j] = 1 if button j affects light i, else 0
    matrix_a = [[0] * num_buttons for _ in range(num_lights)]
    for j, btn_indices in enumerate(buttons):
        for i in btn_indices:
            if 0 <= i < num_lights:
                matrix_a[i][j] = 1
                
    # Gaussian Elimination
    rref_a, rref_b, pivot_cols = rref_binary(matrix_a, target)
    
    # Check for consistency
    # If a row in RREF A is all zeros, but corresponding b is 1, no solution.
    for r in range(num_lights):
        is_all_zeros = all(val == 0 for val in rref_a[r])
        if is_all_zeros and rref_b[r] == 1:
            print("System inconsistent - no solution found.")
            return float('inf')
            
    # Identify free variables
    free_cols = [c for c in range(num_buttons) if c not in pivot_cols]
    
    # Brute force over free variables
    # Since we want minimum presses, and assuming dimension is small enough for AoC.
    # Usually AoC inputs for these types have < 20 free variables.
    
    min_presses = float('inf')
    
    # Generate all combinations for free variables
    # 2^len(free_cols) iterations
    for free_vals in itertools.product([0, 1], repeat=len(free_cols)):
        # Construct solution vector x
        x = [0] * num_buttons
        
        # Assign free variables
        for idx, col_idx in enumerate(free_cols):
            x[col_idx] = free_vals[idx]
            
        # Solve for pivot variables
        # Iterate rows in reverse to back-substitute (though RREF makes it direct)
        # RREF property: The pivot entry is the only non-zero entry in its column.
        # So for a pivot row r with pivot at col c: x[c] + sum(A[r][k]*x[k] for k > c) = b[r]
        # x[c] = b[r] - sum(...)  (but - is + in GF(2))
        
        # Map rows to pivots. rref_binary puts pivots in diagonal-ish form.
        # We need to match pivot columns to the row they were established in.
        # Since we processed cols left to right and inc pivot_row, pivot_cols[i] corresponds to row i.
        
        valid_assignment = True
        
        for r, p_col in enumerate(pivot_cols):
            # Calculate required value for this pivot
            # x[p_col] = b[r] XOR sum(row[k] * x[k] for k in free_cols)
            # Actually sum over all k > p_col. 
            
            row_sum = 0
            # Optimized: only check columns that are 1 in this row (excluding the pivot itself)
            # In RREF, non-zero entries to the right of pivot are coeff for other vars.
            for c in range(p_col + 1, num_buttons):
                if rref_a[r][c] == 1:
                    row_sum ^= x[c]
            
            x[p_col] = rref_b[r] ^ row_sum
            
        # Calculate Hamming weight
        presses = sum(x)
        if presses < min_presses:
            min_presses = presses
            
    return min_presses

def solve_part1(machines):
    """
    Solves the first part of the puzzle.
    """
    print("Solving Part 1...")
    total_presses = 0
    
    for i, machine in enumerate(machines):
        presses = solve_machine(machine)
        if presses == float('inf'):
            # Only print verbose errors if strictly needed, otherwise it clutters output
            pass 
        else:
            total_presses += presses
            
    return total_presses

part1_answer = solve_part1(parsed_input_part1)
print(f"Part 1 Answer: {part1_answer}\n")


# 4. Part 1 Testing
# -----------------------------------------------------------------------------
EXAMPLE_INPUT_PART1_STR = """
[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}
[...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}
[.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}
"""
EXAMPLE_EXPECTED_PART1 = 7

def test_part1():
    print("Running Part 1 example test...")
    if not EXAMPLE_INPUT_PART1_STR.strip():
        print("Skipping test: No example data provided yet.")
        return

    example_raw_data = EXAMPLE_INPUT_PART1_STR.strip().split('\n')
    example_parsed_data = parse_data_part1(example_raw_data)
    result = solve_part1(example_parsed_data)
    
    assert result == EXAMPLE_EXPECTED_PART1, \
        f"Part 1 Example Failed! Expected {EXAMPLE_EXPECTED_PART1}, Got {result}"
    print(f"Test Result: {result} (Expected: {EXAMPLE_EXPECTED_PART1})")
    print("Part 1 example test finished.\n")

# Run the test
test_part1()


# =============================================================================
# >>> START PART 2 (Only after solving Part 1) <<<
# =============================================================================

# 5. Part 2 Data Preprocessing / Parsing
# -----------------------------------------------------------------------------
def parse_data_part2(data_lines):
    """
    Parses the raw input data for Part 2.
    """
    print("Parsing data for Part 2...")
    if not data_lines:
        return []

    # Reuses updated Part 1 parsing which now includes 'joltage'
    return parse_data_part1(data_lines)

parsed_input_part2 = parse_data_part2(raw_data)


# 6. Part 2 Solution Algorithm
# -----------------------------------------------------------------------------
def rref_rational(matrix, vector_b):
    """
    Computes the RREF of [A|b] over Rationals.
    """
    rows = len(matrix)
    if rows == 0:
        return [], [], []
    cols = len(matrix[0])
    
    # Augmented matrix with Fractions
    aug = []
    for r in range(rows):
        row = [Fraction(x, 1) for x in matrix[r]] + [Fraction(vector_b[r], 1)]
        aug.append(row)
        
    pivot_row = 0
    pivot_cols = []
    
    for c in range(cols):
        if pivot_row >= rows:
            break
            
        # Find non-zero pivot
        row_with_val = -1
        for r in range(pivot_row, rows):
            if aug[r][c] != 0:
                row_with_val = r
                break
                
        if row_with_val != -1:
            aug[pivot_row], aug[row_with_val] = aug[row_with_val], aug[pivot_row]
            
            # Normalize pivot row
            pivot_val = aug[pivot_row][c]
            for k in range(c, cols + 1):
                aug[pivot_row][k] /= pivot_val
                
            # Eliminate other rows
            for r in range(rows):
                if r != pivot_row and aug[r][c] != 0:
                    factor = aug[r][c]
                    for k in range(c, cols + 1):
                        aug[r][k] -= factor * aug[pivot_row][k]
            
            pivot_cols.append(c)
            pivot_row += 1
            
    final_matrix = [row[:cols] for row in aug]
    final_b = [row[cols] for row in aug]
    
    return final_matrix, final_b, pivot_cols

def solve_machine_part2(machine):
    target = machine['joltage'] # Use Joltage counters
    buttons = machine['buttons']
    
    num_counters = len(target)
    num_buttons = len(buttons)
    
    # Construct Matrix A
    matrix_a = [[0] * num_buttons for _ in range(num_counters)]
    for j, btn_indices in enumerate(buttons):
        for i in btn_indices:
            if 0 <= i < num_counters:
                matrix_a[i][j] = 1
                
    # RREF over Rationals
    rref_a, rref_b, pivot_cols = rref_rational(matrix_a, target)
    
    # Check consistency
    for r in range(num_counters):
        is_all_zeros = all(val == 0 for val in rref_a[r])
        if is_all_zeros and rref_b[r] != 0:
            return float('inf')

    # Identify free variables
    free_cols = [c for c in range(num_buttons) if c not in pivot_cols]
    
    # Calculate bounds for free variables
    # x_pivot = b_row - sum(A_row_j * x_j for j in free)
    # Since x_pivot >= 0:
    # sum(A_row_j * x_j) <= b_row
    # Since all inputs were non-negative, and we only add,
    # coefficients in RREF can be positive or negative.
    # However, A_row_j > 0 implies an upper bound on x_j.
    
    # We will search. Since problem implies unique or simple solutions,
    # and "button presses" shouldn't be astronomically high for these small targets,
    # we iterate free variables.
    
    # Determine search ranges. 
    # Conservative upper bound: max(target) is usually small (e.g., 12).
    # A single button press adds at least 1 to some counter.
    # So no button can be pressed more than max(target) times roughly.
    max_press_limit = max(target) if target else 0
    
    min_total = float('inf')
    
    # Generate combinations. 
    # Optimization: Sort free cols by their impact on the objective function?
    # Simple brute force with range 0..max_press_limit+1
    
    # Usually few free variables in these AoC problems.
    ranges = [range(max_press_limit + 1) for _ in free_cols]
    
    for free_vals in itertools.product(*ranges):
        x = [Fraction(0, 1)] * num_buttons
        
        # Assign free vars
        for idx, col_idx in enumerate(free_cols):
            x[col_idx] = Fraction(free_vals[idx], 1)
            
        # Solve for pivots
        valid = True
        for r, p_col in enumerate(pivot_cols):
            # x[p_col] = b[r] - sum(A[r][k]*x[k])
            row_sum = Fraction(0, 1)
            for c in range(p_col + 1, num_buttons):
                if rref_a[r][c] != 0:
                    row_sum += rref_a[r][c] * x[c]
            
            val = rref_b[r] - row_sum
            
            # Check constraints
            if val < 0 or val.denominator != 1:
                valid = False
                break
            x[p_col] = val
            
        if valid:
            total = sum(x)
            if total < min_total:
                min_total = total
                
    return min_total

def solve_part2(machines):
    """
    Solves the second part of the puzzle.
    """
    print("Solving Part 2...")
    total_presses = 0
    
    for i, machine in enumerate(machines):
        # Only process if we successfully parsed joltage
        if not machine['joltage']:
            continue
            
        presses = solve_machine_part2(machine)
        if presses != float('inf'):
            total_presses += presses.numerator # It's a Fraction with denom 1
            
    return total_presses

part2_answer = solve_part2(parsed_input_part2)
print(f"Part 2 Answer: {part2_answer}\n")


# 7. Part 2 Testing
# -----------------------------------------------------------------------------
EXAMPLE_INPUT_PART2_STR = EXAMPLE_INPUT_PART1_STR
EXAMPLE_EXPECTED_PART2 = 33

def test_part2():
    print("Running Part 2 example test...")
    if not EXAMPLE_INPUT_PART2_STR.strip():
        print("Skipping test: No example data provided yet.")
        return

    example_raw_data = EXAMPLE_INPUT_PART2_STR.strip().split('\n')
    example_parsed_data = parse_data_part2(example_raw_data)
    result = solve_part2(example_parsed_data)
    
    assert result == EXAMPLE_EXPECTED_PART2, \
        f"Part 2 Example Failed! Expected {EXAMPLE_EXPECTED_PART2}, Got {result}"
    print(f"Test Result: {result} (Expected: {EXAMPLE_EXPECTED_PART2})")
    print("Part 2 example test finished. Joltage optimized!\n")

# Run the Part 2 test
test_part2()

print("\n✨🎄 Join the event https://adventofcode.com/2025, let's code before Christmas")

Loaded 182 lines from 'input.txt'.
First 5 lines: ['[.##......] (0,1,3,4,6,7,8) (1,2,3,5,6,8) (0,1) (3,5,6,7) (2,5,7) (1,2,3,4,5,7,8) (7) (0,1,3) (0,3,7) (1,4,6) {36,63,29,56,28,48,43,52,23}', '[.##....#] (0,1,2,3,4,5,7) (0,2,3,7) (1,3,5,6) (0,1,2,6) (2,3,5,6,7) (2,4) (0,1,2,3,7) (0,2,3,5,6,7) {171,166,199,75,30,65,170,64}', '[.#.#.] (0,2,3,4) (1,3) (0,2) (1,4) {1,27,1,7,20}', '[#...#] (1,4) (2,3) (0,2,3) (1,2,3) (0,1,4) {7,22,11,11,15}', '[##.#.#.] (1,3,4,6) (4,5) (0,1,3,4) (0,4,6) (1,2,3,4) (1,2,4,5) (0,1,4,5,6) (0,3) (0,2,4,5) {46,44,15,39,72,45,31}']

Parsing data...
Solving Part 1...
Part 1 Answer: 477

Running Part 1 example test...
Parsing data...
Solving Part 1...
Test Result: 7 (Expected: 7)
Part 1 example test finished.

Parsing data for Part 2...
Parsing data...
Solving Part 2...
Part 2 Answer: 17970

Running Part 2 example test...
Parsing data for Part 2...
Parsing data...
Solving Part 2...
Test Result: 33 (Expected: 33)
Part 2 example test finished. Joltage optimized!


✨🎄

In [ ]:
# --- AoC 2025 - Day 10: Factory ---

# 0. Configuration and Imports
# -----------------------------------------------------------------------------
import os
import sys
import re
from collections import defaultdict, Counter, deque
import math
import itertools
import heapq

# Define input filename - typically both parts use the same 'input.txt'
INPUT_FILENAME = "input.txt"

# Set up paths for convenience
NOTEBOOK_DIR = os.getcwd() 


# 1. Load Input Data
# -----------------------------------------------------------------------------
def load_input_data(filename):
    """
    Loads input data from a specified file.
    Assumes the input file is in the same directory as the notebook.
    """
    filepath = os.path.join(NOTEBOOK_DIR, filename)
    try:
        with open(filepath, 'r') as f:
            # Read all lines and strip whitespace from each
            return [line.strip() for line in f.readlines()]
    except FileNotFoundError:
        print(f"Error: Input file '{filename}' not found at '{filepath}'")
        return [] # Return an empty list to prevent further errors

# Load the raw data once for both parts
raw_data = load_input_data(INPUT_FILENAME)

if raw_data:
    print(f"Loaded {len(raw_data)} lines from '{INPUT_FILENAME}'.")
    print(f"First 5 lines: {raw_data[:5]}\n")
else:
    print("No data loaded (or file not found). Operations will rely on test data.\n")


# =============================================================================
# >>> START PART 1 (Solve this first!) <<<
# =============================================================================

# 2. Part 1 Data Preprocessing / Parsing
# -----------------------------------------------------------------------------
def parse_data_part1(data_lines):
    """
    Parses the raw input data for Part 1.
    Each line represents a machine.
    Returns a list of machines. Each machine is a dict:
    {
        'target': [0, 1, 1, 0],  # Binary vector for lights
        'buttons': [[3], [1, 3], ...] # List of lists of affected light indices
    }
    """
    print("Parsing data for Part 1...")
    if not data_lines: 
        return []

    machines = []
    
    # Regex to capture the lights part [ ... ] and the buttons part ( ... )
    # Example: [.##.] (3) (1,3) ... { ... }
    
    for line in data_lines:
        if not line: continue
        
        # Split lights section from the rest
        try:
            lights_match = re.search(r'\[([.#]+)\]', line)
            if not lights_match:
                print(f"Warning: No lights found in line: {line}")
                continue
            
            lights_str = lights_match.group(1)
            target_vector = [1 if c == '#' else 0 for c in lights_str]
            
            # Find all button definitions (numbers in parens)
            # Ignoring the joltage part in curly braces
            buttons_section = line[lights_match.end():]
            # Remove joltage part if exists to avoid confusion
            if '{' in buttons_section:
                buttons_section = buttons_section.split('{')[0]
                
            button_matches = re.findall(r'\(([\d,]+)\)', buttons_section)
            buttons = []
            for btn_str in button_matches:
                indices = [int(x) for x in btn_str.split(',')]
                buttons.append(indices)
                
            machines.append({
                'target': target_vector,
                'buttons': buttons
            })
            
        except Exception as e:
            print(f"Error parsing line '{line}': {e}")
            continue

    return machines

parsed_input_part1 = parse_data_part1(raw_data)


# 3. Part 1 Solution Algorithm
# -----------------------------------------------------------------------------
def rref_binary(matrix, vector_b):
    """
    Computes the Reduced Row Echelon Form (RREF) of the augmented matrix [A|b] over GF(2).
    matrix: list of lists (rows), represents A
    vector_b: list (column vector), represents b
    
    Returns: 
    - rref_matrix: The transformed matrix A (list of lists)
    - transformed_b: The transformed vector b
    - pivot_cols: List of column indices that contain a pivot
    """
    rows = len(matrix)
    if rows == 0:
        return [], [], []
    cols = len(matrix[0])
    
    # Create augmented matrix as a list of lists (mutable)
    aug = [row[:] + [b_val] for row, b_val in zip(matrix, vector_b)]
    
    pivot_row = 0
    pivot_cols = []
    
    for c in range(cols):
        if pivot_row >= rows:
            break
            
        # Find a row with a 1 in this column, starting from pivot_row
        row_with_one = -1
        for r in range(pivot_row, rows):
            if aug[r][c] == 1:
                row_with_one = r
                break
        
        if row_with_one != -1:
            # Swap rows
            aug[pivot_row], aug[row_with_one] = aug[row_with_one], aug[pivot_row]
            
            # Eliminate 1s in other rows for this column
            for r in range(rows):
                if r != pivot_row and aug[r][c] == 1:
                    # Row XOR
                    for k in range(c, cols + 1):
                        aug[r][k] ^= aug[pivot_row][k]
            
            pivot_cols.append(c)
            pivot_row += 1
            
    # Separate A and b back
    final_matrix = [row[:cols] for row in aug]
    final_b = [row[cols] for row in aug]
    
    return final_matrix, final_b, pivot_cols

def solve_machine(machine):
    """
    Solves for the minimum button presses for a single machine.
    Ax = b over GF(2).
    A: columns are buttons, rows are lights.
    b: target lights.
    """
    target = machine['target']
    buttons = machine['buttons']
    
    num_lights = len(target)
    num_buttons = len(buttons)
    
    # Construct Matrix A
    # A[i][j] = 1 if button j affects light i, else 0
    matrix_a = [[0] * num_buttons for _ in range(num_lights)]
    for j, btn_indices in enumerate(buttons):
        for i in btn_indices:
            if 0 <= i < num_lights:
                matrix_a[i][j] = 1
                
    # Gaussian Elimination
    rref_a, rref_b, pivot_cols = rref_binary(matrix_a, target)
    
    # Check for consistency
    # If a row in RREF A is all zeros, but corresponding b is 1, no solution.
    for r in range(num_lights):
        is_all_zeros = all(val == 0 for val in rref_a[r])
        if is_all_zeros and rref_b[r] == 1:
            print("System inconsistent - no solution found.")
            return float('inf')
            
    # Identify free variables
    free_cols = [c for c in range(num_buttons) if c not in pivot_cols]
    
    # Brute force over free variables
    # Since we want minimum presses, and assuming dimension is small enough for AoC.
    # Usually AoC inputs for these types have < 20 free variables.
    
    min_presses = float('inf')
    
    # Generate all combinations for free variables
    # 2^len(free_cols) iterations
    for free_vals in itertools.product([0, 1], repeat=len(free_cols)):
        # Construct solution vector x
        x = [0] * num_buttons
        
        # Assign free variables
        for idx, col_idx in enumerate(free_cols):
            x[col_idx] = free_vals[idx]
            
        # Solve for pivot variables
        # Iterate rows in reverse to back-substitute (though RREF makes it direct)
        # RREF property: The pivot entry is the only non-zero entry in its column.
        # So for a pivot row r with pivot at col c: x[c] + sum(A[r][k]*x[k] for k > c) = b[r]
        # x[c] = b[r] - sum(...)  (but - is + in GF(2))
        
        # Map rows to pivots. rref_binary puts pivots in diagonal-ish form.
        # We need to match pivot columns to the row they were established in.
        # Since we processed cols left to right and inc pivot_row, pivot_cols[i] corresponds to row i.
        
        valid_assignment = True
        
        for r, p_col in enumerate(pivot_cols):
            # Calculate required value for this pivot
            # x[p_col] = b[r] XOR sum(row[k] * x[k] for k in free_cols)
            # Actually sum over all k > p_col. 
            
            row_sum = 0
            # Optimized: only check columns that are 1 in this row (excluding the pivot itself)
            # In RREF, non-zero entries to the right of pivot are coeff for other vars.
            for c in range(p_col + 1, num_buttons):
                if rref_a[r][c] == 1:
                    row_sum ^= x[c]
            
            x[p_col] = rref_b[r] ^ row_sum
            
        # Calculate Hamming weight
        presses = sum(x)
        if presses < min_presses:
            min_presses = presses
            
    return min_presses

def solve_part1(machines):
    """
    Solves the first part of the puzzle.
    """
    print("Solving Part 1...")
    total_presses = 0
    
    for i, machine in enumerate(machines):
        presses = solve_machine(machine)
        if presses == float('inf'):
            print(f"Machine {i} impossible to configure.")
        else:
            total_presses += presses
            
    return total_presses

part1_answer = solve_part1(parsed_input_part1)
print(f"Part 1 Answer: {part1_answer}\n")


# 4. Part 1 Testing
# -----------------------------------------------------------------------------
EXAMPLE_INPUT_PART1_STR = """
[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}
[...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}
[.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}
"""
EXAMPLE_EXPECTED_PART1 = 7

def test_part1():
    print("Running Part 1 example test...")
    if not EXAMPLE_INPUT_PART1_STR.strip():
        print("Skipping test: No example data provided yet.")
        return

    example_raw_data = EXAMPLE_INPUT_PART1_STR.strip().split('\n')
    example_parsed_data = parse_data_part1(example_raw_data)
    result = solve_part1(example_parsed_data)
    
    assert result == EXAMPLE_EXPECTED_PART1, \
        f"Part 1 Example Failed! Expected {EXAMPLE_EXPECTED_PART1}, Got {result}"
    print(f"Test Result: {result} (Expected: {EXAMPLE_EXPECTED_PART1})")
    print("Part 1 example test finished.\n")

# Run the test
test_part1()


# =============================================================================
# >>> START PART 2 (Only after solving Part 1) <<<
# =============================================================================

# 5. Part 2 Data Preprocessing / Parsing
# -----------------------------------------------------------------------------
def parse_data_part2(data_lines):
    """
    Parses the raw input data for Part 2.
    """
    print("Parsing data for Part 2...")
    if not data_lines:
        return []

    # Usually reuses Part 1 parsing
    return parse_data_part1(data_lines)

parsed_input_part2 = parse_data_part2(raw_data)


# 6. Part 2 Solution Algorithm
# -----------------------------------------------------------------------------
def solve_part2(data):
    """
    Solves the second part of the puzzle.
    """
    print("Solving Part 2...")
    
    # --- Part 2 Algorithm Here ---

    return "Part 2 Solution Not Implemented Yet"

part2_answer = solve_part2(parsed_input_part2)
print(f"Part 2 Answer: {part2_answer}\n")


# 7. Part 2 Testing
# -----------------------------------------------------------------------------
EXAMPLE_INPUT_PART2_STR = EXAMPLE_INPUT_PART1_STR
EXAMPLE_EXPECTED_PART2 = 0

def test_part2():
    print("Running Part 2 example test...")
    if not EXAMPLE_INPUT_PART2_STR.strip():
        print("Skipping test: No example data provided yet.")
        return

    example_raw_data = EXAMPLE_INPUT_PART2_STR.strip().split('\n')
    example_parsed_data = parse_data_part2(example_raw_data)
    result = solve_part2(example_parsed_data)
    
    # assert result == EXAMPLE_EXPECTED_PART2, \
    #     f"Part 2 Example Failed! Expected {EXAMPLE_EXPECTED_PART2}, Got {result}"
    print(f"Test Result: {result} (Expected: {EXAMPLE_EXPECTED_PART2})")
    print("Part 2 example test finished.\n")

# Run the Part 2 test
# test_part2()

print("\n✨🎄 Join the event https://adventofcode.com/2025, let's code before Christmas")